# Introduction

For this analysis, I will be using a dataset containing information about  listings on Airbnb. The source of the data is Inside AirBnb which is a set of tools and data that allows you to explore information about listings on Airbnb.  The analysis will use d3 graphs to visualize and answer questions related to the different price ranges for listings in different neighbourhood, the relationship between room type and price and which neighbourhood will likely have listings based on a user preferences. My sources for this analysis include:
 1. https://www.kaggle.com/dgomonov/data-exploration-on-nyc-airbnb
 2. https://www.d3-graph-gallery.com/graph/violin_basicHist.html
 3. https://www.d3-graph-gallery.com/graph/barplot_ordered.html
 4. http://insideairbnb.com/get-the-data.html
 5. https://www.d3-graph-gallery.com/boxplot.html
 6. https://data.cityofnewyork.us/City-Government/Borough-Boundaries/tqmj-j8zm
 7. https://www.d3-graph-gallery.com/choropleth.html

### Data definition:

### AB_NYC_2019.csv:
The AB_NYC_2019.csv is a table dataset with 16 columns or attributes. For this analysis, I’m focusing on 6 attributes: The price, neighbourhood_group, room_type,availability_365, longitude, and latitude. The dataset is a static dataset. It is important to note that the longitude and latitude are considered spatial data for this analysis.  Below is a summary for the attributes in the dataset.

| Attribute/Column | Attribute Type | Possible Values | Description |
| :--- | :---: | :---: | :--- |
| id | Categorical | 3647| The listing id  |
| name | Categorical | Skylit Midtown Castle | The name of the listing on Airbnb |
| host_id | Categorical | 2787 | the host id |
| host_name | Categorical | Chris | A string representing the name of the host  |
| neighbourhood_group | Categorical | Queens | A string represnting the name of the area of the listing |
| neighbourhood | Categorical | Midtown	 | A string representing the location of the listing. |
| latitude | Ordered | 40.7353 | The latitude of the listing |
| longitude | Ordered | -74.54353 | The longitude of the listing |
| room_type | Categorical | Entire Home | The type of listing. It can be either entire house, private room or shared room. |
| price | Ordered | 50 | The price per night for the listing |
| minimum_nights | Ordered | 1 | The minimum number of nights the listing can be rented for |
| number_of_reviews | Ordered | 1 | The number of reviews for the listing on Airbnb |
| last_review | Categorical | 2017-10-05 | The date for the last review posted on Airbnb |
| reviews_per_month | Ordered | 1.5 | The average number of reviews per month|
| availability_365 | Ordered | 39 | number of days when listing is available for booking |
| calculated_host_listings_count| Ordered | 1 | The number of listings for the host|

### att.csv:
The att.csv is a spatial dataset with 3 columns or attributes. Below is a summary for the attributes in the dataset.

| Attribute/Column | Attribute Type | Possible Values | Description |
| :--- | :---: | :---: | :--- |
| attraction | Categorical | "Bronx Zoo" | The name of the attraction spot |
| latitude | Ordered | 40.689247 | The latitude of the attraction spot | 
| longitude | Ordered | -74.044502 | The longitude of the attraction spot |

### borough.geojson:
A geometry dataset with Categorical items and Ordered positions. Each item in the dataset represents a neighbourhood_group in the AB_NYC_2019.csv.

# Analysis

### User Tasks:
__1. Does Brooklyn have the most listings priced under $100 per night__ <br><br>
__2. Does the Bronx have the cheapest price range?__ <br><br>
__3. Show the price density and distribution for each room type to find which type of room is generally cheaper.__<br><br> 
__4. Which Neighbourhood near the Statue of Liberty  have the most private room listings with price per night under 50 and availability under 3 month__<br><br>

### Actions and Targets:
1. Task 1: <br>
Since the target and the location of this task is already known (Brooklyn). Therefore, the action type is Search, and the search type is __lookup__. The location with largest number of listings priced under $100 per night is the extremes, so the target here is is __Extremes__.<br> 
2. Task 2:<br>
Since the target is known, and location is given(cheapest price range), the action is __lookup__.The cheapest price range is an __extreme__.<br> 
3. Task 3:<br> 
Since I'm analysing the data to guide the user(find which room type is genearly cheaper) so the action is __present__.The target of the user is to find __distrubtion__ of the prices of single attribute which is the room type.<br>
4. Task 4: <br>
Since the target and the location(private room listings with price per night under 50 and availability under 3 month) are known, the action is __lookup__.The location of the target is __extreme__.

### Plots:

In [1]:
%%javascript

require.config({
    paths: {
        d3v5: 'https://d3js.org/d3.v5.min',
        d3v4: 'https://d3js.org/d3.v4.min'
    }
});

<IPython.core.display.Javascript object>

In [8]:
%%javascript

require(['d3v4'], function(d3) {

  function cvert(d) {
    return {
      neighbourhood_group: d.neighbourhood_group,
      price: parseFloat(d.price),
      room_type: d.room_type,
      ava: parseFloat(d.availability_365)
    }
  }

  d3.csv("AB_NYC_2019.csv", cvert, function(data) {
      var filtered = data.filter(function(d) {
  return d.price < 100  }) 
      
    

    var margin = {
        top: 30,
        right: 30,
        bottom: 70,
        left: 60
      },
      width = 650 - margin.left - margin.right,
      height = 600 - margin.top - margin.bottom;

    var svg = d3.select(element.get(0))
      .append("svg")
      .attr("width", width + margin.left + margin.right)
      .attr("height", height + margin.top + margin.bottom)
      .append("g")
      .attr("transform",
        "translate(" + margin.left + "," + margin.top + ")");



    var nest = d3.nest()
      .key(function(d) {
        return d.neighbourhood_group;
      })
      .rollup(function(values) {
        return values.length
      })
      .entries(filtered)
      .sort(function(a, b) {
        return d3.descending(a.value, b.value);
      });




    var x = d3.scaleBand()
      .range([0, width])
      .domain(nest.map(function(d) {
        return d.key
      }))
      .padding(0.2);
    svg.append("g")
      .attr("transform", "translate(0," + height + ")")
      .call(d3.axisBottom(x))
      .selectAll("text")
      .attr("transform", "translate(-10,0)rotate(-45)")
      .style("text-anchor", "end")
      .style("font-size", "13px");

    var maxy = d3.max(nest, function(d) {
      return d.value
    }) + 1000;

    var y = d3.scaleLinear()
      .domain([0, maxy])
      .range([height, 0]);
    svg.append("g")
      .call(d3.axisLeft(y))
      .style("font-size", "11px");;

var colors = d3.scaleOrdinal(d3.schemeCategory10);
    svg.append("text")
      .attr("transform",
        "translate(" + (width / 2) + " ," +
        (height + margin.top + 35) + ")")
      .style("text-anchor", "middle")
      .text("Neighbourhood")
      .style("font", "bold 15px sans-serif");

    svg.append("text")
      .attr("transform", "rotate(-90)")
      .attr("y", 0 - margin.left - 4)
      .attr("x", 0 - (height / 2))
      .attr("dy", "1em")
      .style("font", "bold 15px sans-serif")
      .style("text-anchor", "middle")
      .text("Number of Listings");

    svg.append("text")
      .attr("y", 0)
      .attr("x", width / 2)
      .style("font", "bold 20px sans-serif")
      .style("text-anchor", "middle")
      .text("Plot 1: Number of Listings priced under 100 by Neighbourhood ");


    svg.selectAll("mybar")
      .data(nest)
      .enter()
      .append("rect")
      .attr("x", function(d) {
        return x(d.key);
      })
      .attr("y", function(d) {
        return y(d.value);
      })
      .attr("width", x.bandwidth())
      .attr("height", function(d) {
        return height - y(d.value);
      })
      .attr("fill", (d, i) => colors(i))


  });
});


<IPython.core.display.Javascript object>

Plot 1 is a bar plot for the number of listings for Neighbourhood. It clearly shows that Brooklyn have the most listings with price per night under 100. Below is summary for idioms, channels, and marks.<br> 
__Data__: The x axis shows the neighbourhood which is a key categorical attrubite. The y axis shows the number of listings which is an ordered quantitive value. <br> 
__Mark__: vertical line <br>
__Channels__: For the number of listings, a length channel was used. For Neighbourhood name, one spatial region was created for each Neighbourhood name.  Each region was aligned vertically. the spatial regions were ordered by the number of listings. A color hue channel was also used for the Neighbourhood name.<br>

__Methodology__:
* bar plot effectively convey both the key categorical attrubite and ordered quantitive value at the same time. 
* the bar plot allows me to use two channels for each key categorical attrubite. A bar plot can utilize a spatial region and color hue channels which two of the most effective channels based Munzner's what-why-how analysis scale.
* Bar plots allows me to order each spatial region by the quantitive value easily. The ordering of the spatial region helps with finding the extremes.<br>    

__Visualization__:<br>
Because the target and action here is to lookup extremes, I had to draw a plot to show the extremes, and because this extremes is based on quantitative value(the number of listings), a bar plot is suitable. The ordering of the spatial regions from longest to shortest is beneficial for the user since the user is trying to find if Brooklyn have the most listings under 100. The user just needs to confirm if the spatial region for Brooklyn is first.

In [5]:
%%javascript

require(['d3v4'], function(d3) {

  function cvert(d) {
    return {
      neighbourhood_group: d.neighbourhood_group,
      price: parseFloat(d.price),
    }
  }

  d3.csv("AB_NYC_2019.csv", cvert, function(data) {
    var margin = {
      top: 20,
      right: 40,
      bottom: 40,
      left: 50
    };
    var width = 650 - margin.left - margin.right;
    var height = 600 - margin.top - margin.bottom;

    var svg = d3.select(element.get(0))
      .append("svg")
      .attr("width", width + margin.left + margin.right)
      .attr("height", height + margin.top + margin.bottom)
      .append("g")
      .attr("transform",
        "translate(" + margin.left + "," + margin.top + ")");
      var sumstat = d3.nest() // nest function allows to group the calculation per level of a factor
    .key(function(d) { return d.neighbourhood_group;})
    .rollup(function(d) {
      var q1 = d3.quantile(d.map(function(g) { return g.price;}).sort(d3.ascending),.25)
      var median = d3.quantile(d.map(function(g) { return g.price;}).sort(d3.ascending),.5)
      var q3 = d3.quantile(d.map(function(g) { return g.price;}).sort(d3.ascending),.75)
      var interQuantileRange = q3 - q1
      var min = (q1 - 0.5 * interQuantileRange) > (d3.min(d.map(function(g) { return g.price;}))) ?  (q1 - .5 * interQuantileRange) : d3.min(d.map(function(g) { return g.price;}))
      var max = (q3 + .5 * interQuantileRange) < (d3.max(d.map(function(g) { return g.price;}))) ?  (q3 + 0.5 * interQuantileRange) : d3.max(d.map(function(g) { return g.price;}))
      var range = max - min
      
      
      return({q1: q1, median: median, q3: q3, interQuantileRange: interQuantileRange, min: min , max: max,range:range})
    })
    .entries(data)
      
       sumstat.sort(function(x, y){
   return d3.ascending(x.value.range, y.value.range);
})
      
      
 var groups = []
sumstat.forEach(function (a) {
    groups.push(a.key);
});
var x = d3.scaleBand()
    .range([ 0, width ])
    .domain(groups)
    .paddingInner(1)
    .paddingOuter(.5)
  svg.append("g")
    .attr("transform", "translate(0," + height + ")")
    .call(d3.axisBottom(x))
  var y = d3.scaleLinear()
    .domain([0, d3.max(sumstat, d => d.value.max)+50])
    .range([height, 0])
  svg.append("g").call(d3.axisLeft(y))
      var boxWidth = 100
      
      var colors = d3.scaleOrdinal(d3.schemeCategory10);
  svg
    .selectAll("vertLines")
    .data(sumstat)
    .enter()
    .append("line")
      .attr("x1", function(d){return(x(d.key))})
      .attr("x2", function(d){return(x(d.key))})
      .attr("y1", function(d){return(y(d.value.min))})
      .attr("y2", function(d){return(y(d.value.max))})
      .attr("stroke", (d, i) => colors(i))
      .style("stroke-width", 10)
    

  // Show the median
  
      
      svg.append("text")
      .attr("transform",
        "translate(" + (width / 2) + " ," +
        (height + margin.top + 15) + ")")
      .style("text-anchor", "middle")
      .text("Neighbourhood")
      .style("font", "bold 15px sans-serif");

    svg.append("text")
      .attr("transform", "rotate(-90)")
      .attr("y", 0 - margin.left - 4)
      .attr("x", 0 - (height / 2))
      .attr("dy", "1em")
      .style("font", "bold 15px sans-serif")
      .style("text-anchor", "middle")
      .text("Price");

    svg.append("text")
      .attr("y", 0)
      .attr("x", width / 2)
      .style("font", "bold 20px sans-serif")
      .style("text-anchor", "middle")
      .text("Plot 2: Price ranges per Neighbourhood  ");

    
  });
})

<IPython.core.display.Javascript object>

Plot 2 shows the price ranges in each Neighbourhood . The plot shows Bronx has the cheapest price range. <br> 
__Data__: The x axis shows the neighbourhood name which is a key categorical attrubite. The y axis shows the prices of listings which is a ordered  quantitive value. 
<br> __Mark__: lines.<br> 
__Channels__: The price range is  represented by the vertical length of the lines.  A color hue and spatial region channel were also used for each Neighbourhood name. <br> 
<br>__Methodolgy__:<br>
For this plot, I decided to use one component of a boxplot which is the whisker. The whisker represent a price range using a vertical length. The whisker is acceptable representation of the price range because it doesnt take into account the outliers or suspected outliers in the prices. Each Neighbourhood price range is differentiated using a color hue.   
<br>__Visualizaon__:<br>
Since the user is only intersted in the price range, the quartiles box of boxplot is not visualized and only the whisker component is shown. The user can instantly recognize the price range belonging to Bronx neighbourhood by looking at the color of the line. The user can get an idea about the price range by looking at where the vertical line starts and ends.  

In [2]:
%%javascript

require(['d3v4'], function(d3) {

  function cvert(d) {
    return {
      neighbourhood_group: d.neighbourhood_group,
      price: parseFloat(d.price),
      type: d.room_type
    }
  }

  d3.csv("AB_NYC_2019.csv", cvert, function(data) {
    var margin = {
      top: 20,
      right: 40,
      bottom: 40,
      left: 50
    };
    var width = 650 - margin.left - margin.right;
    var height = 600 - margin.top - margin.bottom;

    var svg = d3.select(element.get(0))
      .append("svg")
      .attr("width", width + margin.left + margin.right)
      .attr("height", height + margin.top + margin.bottom)
      .append("g")
      .attr("transform",
        "translate(" + margin.left + "," + margin.top + ")");

    var y = d3.scaleLinear()
      .domain([0, 550])
      .range([height, 0])
    svg.append("g").call(d3.axisLeft(y)).style("font-size", "13px");
    var area = d3.map(data, function(d) {
      return d.type;
    }).keys();
    var x = d3.scaleBand()
      .range([0, width])
      .domain(area)
      .padding(0.03)
    svg.append("g")
      .attr("transform", "translate(0," + height + ")")
      .call(d3.axisBottom(x))
      .style("font-size", "13px");


    var histogram = d3.histogram()
      .domain(y.domain())
      .thresholds(y.ticks(20))
      .value(d => d)

    var sumstat = d3.nest()
      .key(function(d) {
        return d.type;
      })
      .rollup(function(d) {
        var input = d.map(function(g) {
          return g.price;
        })
        var bins = histogram(input)
        return (bins)
      })
      .entries(data)

    var maxNum = 0
    for (var i in sumstat) {
      var allBins = sumstat[i].value
      var lengths = allBins.map(function(a) {
        return a.length;
      })
      var longuest = d3.max(lengths)
      if (longuest > maxNum) {
        maxNum = longuest
      }
    }


    var xNum = d3.scaleLinear()
      .range([0, x.bandwidth()])
      .domain([-maxNum, maxNum])

    svg.append("text")
      .attr("y", 0)
      .attr("x", width / 2)
      .style("font", "bold 20px sans-serif")
      .style("text-anchor", "middle")
      .text("Plot 2: Density & distribution of prices for each room type");
    
var colors = d3.scaleOrdinal(d3.schemeCategory10);

    svg.append("text")
      .attr("transform",
        "translate(" + (width / 2) + " ," +
        (height + margin.top + 15) + ")")
      .style("text-anchor", "middle")
      .text("Room Type")
      .style("font", "bold 15px sans-serif");
    svg.append("text")
      .attr("transform", "rotate(-90)")
      .attr("y", 0 - margin.left)
      .attr("x", 0 - (height / 2))
      .attr("dy", "1em")
      .style("text-anchor", "middle")
      .text("Price($)")
      .style("font", "bold 15px sans-serif");

    svg.selectAll("myViolin")
      .data(sumstat)
      .enter()
      .append("g")
      .attr("transform", function(d) {
        return ("translate(" + x(d.key) + " ,0)")
      })
      .append("path")
      .datum(function(d) {
        return (d.value)
      })
      .style("stroke", "none")
      .style("fill", (d, i) => colors(i))
      .attr("d", d3.area()
        .x0(function(d) {
          return (xNum(-d.length))
        })
        .x1(function(d) {
          return (xNum(d.length))
        })
        .y(function(d) {
          return (y(d.x0))
        })
        .curve(d3.curveStep)
      )
  });
})

<IPython.core.display.Javascript object>

Plot 3 shows the price density and distrubtion through a violin plot for each room type. Most prices for private rooms lies near the 50 dollar mark while for Entire home/apt, the prices lies near 150 mark. The shared room listings are considered the cheapest since most of shared room listing lie at the 25 mark. <br> 
__Data__: The x axis shows the room type which is a key categorical attribute. The y axis shows the prices of listings which is a ordered quantitive value.
<br>__Mark__: Areas.<br> 
__Channels__: Vertical length, horzionatl length, color hues,curvature, and area size. <br> 
<br>__Methodolgy__:<br>

* The vase violin allows me to represent both the price distrubtion and density using a vertical and horziontal length channels.
* The area for each room type is differentiated using a color hue channel.

<br>__Visualizaon__:<br>
Each area color will help the user know the room type. The user can know which room type is generaly the cheapest by looking at the horziontal and vertical length of each area. For this graph, the shared room is genearly the cheapest because the green area have its maximum​ horziontal length at the lowest vertical point between the three areas.   



In [12]:

%%HTML
<form>
<p>Price per night:</p>
  <input class="but" type="radio" name="price" value="1" checked > less than 50
  <input class="but" type="radio" name="price" value="2"> between 50 and 100<br>
  <input class="but" type="radio" name="price" value="3"> between 101 and 150 
<input class="but" type="radio" name="price" value="4"> between 151 and 200<br> 
 <input class="but" type="radio" name="price" value="5"> between 201 and 250 
<input class="but" type="radio" name="price" value="6"> more than 250<br>

<p>Room Type:</p>
  <input class="but" type="radio" name="type" value="p" checked> Private room

  <input class="but" type="radio" name="type" value="e"> Entire home
<br>
  <input class="but" type="radio" name="type" value="s"> Shared room
<br>
<p>Availability per year:</p>
  <input class="but" type="radio" name="ava" value="f1" checked> Less than 3 month

  <input class="but" type="radio" name="ava" value="f2"> between 3 and 6 month 
<br>
  <input class="but" type="radio" name="ava" value="f3"> between 6 and 9 month
    <input class="but" type="radio" name="ava" value="f4"> between 9 and 12 month
<br>
</form>
<svg id="NYCmap">

</svg>

In [13]:
%%javascript

require(['d3v4'], function(d3) {

  function cvert1(d) {
    return {
      neighbourhood_group: d.neighbourhood_group,
      lat: parseFloat(d.latitude),
      long: parseFloat(d.longitude),
      price:parseFloat(d.price),
        type: d.room_type,
        ava: parseFloat(d.availability_365)
     
    }
  }

  function cvert2(d) {
    return {
      att: d.attraction,
      lat: parseFloat(d.lat),
      long: parseFloat(d.long),
    }
  }
  d3.csv("AB_NYC_2019.csv", cvert1, function(data1) {
    d3.csv("att.csv", cvert2, function(data2) {
    

 
       
      //Width and height
      var w = 800;
      var h = 650;
     
      
      //Define map projection
      var nycProj = d3.geoOrthographic() //NYC
              .rotate([74.006,-40.7128])
              .translate([w/2, h/2.5])
              .scale(70000);
    
      //Define path generator
      var nycPathFn = d3.geoPath()
               .projection(nycProj);

      //Locate SVG element
      var svgSelection = d3.select("#NYCmap")
            .attr("width", w)
            .attr("height", h);
            var gg = []
       
     data2.forEach(function(item){
           
           var j = {type: "Feature", properties: item.att, geometry: {type: "Point", coordinates:[item.long,item.lat]}};
          gg.push(j)
       })
       

      //Load in GeoJSON data
      d3.json("borough.geojson", function(boros) {
        
        //Bind data and create group with text and one path per GeoJSON feature
        var entrySelection = svgSelection.selectAll("g")
                                .data(boros.features).enter()
                                .append("g")
        entrySelection.selectAll(".poic")
           .data(gg).enter()
           .append("circle")
           .classed("poic", true)
           .attr("r", 4.5)
           .attr("cx", d => nycProj(d.geometry.coordinates)[0])
           .attr("cy", d => nycProj(d.geometry.coordinates)[1])
           .attr("fill","green").append("title").text(d=>d.properties)
        
            
            ;
         var filtered = data1.filter(function(d) {
  return d["price"] < 50 && d["type"] == "Private room" && d["ava"] < 30})
          var nest = d3.nest()
      .key(function(d) {
        return d.neighbourhood_group;
      })
      .rollup(function(values) {
        return values.length
      })
      .entries(filtered)
      
         var k = {}
        nest.forEach(function(element) {
        k[element.key] = element.value;
});
           
      
        
       var colorScale = d3.scaleThreshold()
  .domain([5, 25,100,400,800,1200,2400,3600,4800,6000])
  .range(["#fff5f0","#fee3d6","#fdc9b4","#fcaa8e","#fc8a6b","#f9694c","#ef4533","#d92723","#bb151a","#970b13","#67000d"])         
        entrySelection
           .append("path")
           .attr("d", nycPathFn)
           .style("stroke", "black").attr("fill", function (d) {
           
        d.total = k[d.properties.boro_name] || 0;
        return colorScale(d.total);
      })
          .each(d => {d.centroid = nycPathFn.centroid(d)})
           ;
          entrySelection          
          .append("text")
          .text(d => d.properties.boro_name)
          .attr("text-anchor","middle")
          .attr("x",  d => d.centroid[0])
          .attr("y", d => d.centroid[1])
            ;
          var ext_color_domain = [0,5, 25,100,400,800,1200,2400,3600,4800];
            var legend_labels = ["<5", "25","100","400","800","1200","2400","3600","4800",">6000"];
         var legend = svgSelection.selectAll("g.legend")
  .data(ext_color_domain)
  .enter().append("g")
  .attr("class", "legend");

  var ls_w = 20, ls_h = 20;

  legend.append("rect")
  .attr("x", 20)
  .attr("y", function(d, i){ return h/2 - (i*ls_h) - 2*ls_h;})
  .attr("width", ls_w)
  .attr("height", ls_h)
  .style("fill", function(d, i) { return colorScale(d); })
  .style("opacity", 0.8);

          
  legend.append("text")
  .attr("x", 50)
  .attr("y", function(d, i){ return h/2 - (i*ls_h) - ls_h - 4;})
  .text(function(d, i){ return legend_labels[i]; });
       
          var legend2 = svgSelection.append("g")
      .attr("font-family", "sans-serif")
      .attr("font-size", 10)
      .attr("text-anchor", "start")
      .attr("transform", function(d, i) {
        return "translate(0," + i * 20 + ")";
      });

    legend2.append("circle")
      .attr("cx", 30)
      .attr("cy", 320)
      .attr("r", 4.5)
      .attr("fill", "green");

    legend2.append("text")
      .attr("x", 35)
      .attr("y", 320)
      .style("font-size", "13px")
      .attr("dy", "0.32em")
      .text("Attraction Point")
          d3.selectAll("form")
        .on("click", function() {
           var radios = d3.selectAll('input')
     var t = {}
           radios._groups[0].forEach(function (a) {
    if(a.checked == true && a.name == "price" ){
        t["pv"] = a.value
    };
                if(a.checked == true && a.name == "type" ){
        t["tv"] = a.value
    };
               if(a.checked == true && a.name == "ava" ){
        t["av"] = a.value
    };
});
          
           if (t.pv == "1"){
                var filtered = data1.filter(function(d) {
  return d["price"] < 50 });
               
           }else if(t.pv == "2"){
                     var filtered = data1.filter(function(d) {
  return d["price"] >= 50 && d["price"] <= 100 }) 
               
           }else if(t.pv == "3"){
               var filtered = data1.filter(function(d) {
  return d["price"] >= 101 && d["price"] <= 150 })
               
           }else if(t.pv == "4"){
               var filtered = data1.filter(function(d) {
  return d["price"] >= 151 && d["price"] <= 200 })
               
           }else if(t.pv == "5"){
               var filtered = data1.filter(function(d) {
  return d["price"] >= 201 && d["price"] <= 250 })
               
           }else if(t.pv == "6"){
               var filtered = data1.filter(function(d) {
  return d["price"] > 250  })
               
           }
           
           if(t.tv == "p"){
               var filtered2 = filtered.filter(function(d) {
  return d["type"] == "Private room"  })
               
           }else if(t.tv == "e"){
               var filtered2 = filtered.filter(function(d) {
  return d["type"] == "Entire home/apt"  })
               
           }else if(t.tv == "s"){
               var filtered2 = filtered.filter(function(d) {
  return d["type"] == "Shared room"  })
               
           }
           
            if(t.av == "f1"){
               var filtered3 = filtered2.filter(function(d) {
  return d["ava"] < 90 })
               
           }else if(t.av == "f2"){
               var filtered3 = filtered2.filter(function(d) {
  return ((d["ava"] >= 90) && (d["ava"] <180))   })
               
           }else if(t.av == "f3"){
               var filtered3 = filtered2.filter(function(d) {
  return ((d["ava"] >= 180) && (d["ava"] <270))   })
               
           }else if(t.av == "f4"){
               var filtered3 = filtered2.filter(function(d) {
  return d["ava"] >= 270    })
               
           }
           
         
       var nest = d3.nest()
      .key(function(d) {
        return d.neighbourhood_group;
      })
      .rollup(function(values) {
        return values.length
      })
      .entries(filtered3)
      
         var k = {}
        nest.forEach(function(element) {
        k[element.key] = element.value;
});
        
        entrySelection
           .append("path")
           .attr("d", nycPathFn)
           .style("stroke", "black")
          .attr("fill", function (d) {
           
        d.total = k[d.properties.boro_name] || 0;
        return colorScale(d.total);
      }).each(d => {d.centroid = nycPathFn.centroid(d)})
            entrySelection          
          .append("text")
          .text(d => d.properties.boro_name)
          .attr("text-anchor","middle")
          .attr("x",  d => d.centroid[0])
          .attr("y", d => d.centroid[1])
            ;
          
           
        }) 
          
          
        
      });
  });
});
})

<IPython.core.display.Javascript object>

Plot 4 shows a choropleth map of new york city. the plot indicates that the closest neighbourhood to Statue of Liberty with  the most private room listings with price under 50 with availability of less than 3 month is Brooklyn.  <br> 
__Data__:  geometry data of neighbourhood, listings and attractions. number of listings as quantitative attribute of each neighbourhood.<br> 
__Marks__: Area and points.<br> 
__Channels__: shape for each neighbourhood and color Luminance for quantitative attribute. position and color hue for the attraction points<br> 
__Manuiplation__:Reduce the items(listings) based on the selected criteria .<br> 
<br>__Originality__:<br>
The orgiginality of my choropleth map comes from the fact that the quantitative attribute​(number of listings) is dervied from a combination of ordered attributes(price and availability) and categorical attributes(room type)​. Furthermore, points which typicaly used in scatter plots is also utilized  <br>
<br>__Methodolgy__:<br>
* Since I wanted to show the number of listings(quantitative), the color Luminance of choropleth map is used.  the number of listings is represented using white to red 11 level sequence with monotonically decreasing luminance. 
* The choropleth map helps derive shaps from the geometric data of the neighbourhood to present the data in a way that a user can comprehend . 
* a point is used for each attraction point. The distance between the point of the attraction and the shape neighbourhood shows the relative distance between the  neighbourhood and the attraction spot. The attraction points uses a green color to contrast with the white to red scale used for the number of listings. 

__Visualizaon__:<br>
While the task for this plot is similar to the task for plot 1, choropleth map was chosen instead of a bar plot since the task involves multiple attributes.  The user for this task wants to find the neighbourhood near the Statue of Liberty  with most lsitings with multiple criteria. In order to locate the Statue of Liberty point, the user needs to hover over the green points since it will clearly indicate the name of the attraction point. To find the neighbourhood near the attraction points the user will need to check the name of the shapes around the attraction point. Finally to know which neighbourhood shape has the most listings, the user will check the shape with darkest red color.     

# Conclusion 
For most of my plots the most effective channels according to munzier analysis were used. The spatial region and color hue channel were used in plot 1 and 2 to help the user find the neighboorhood easily on the plot. For plot 2, the whisker might not be a good representation for the price range for the user if the prices are skewed. I tried to overcome that by taking into account only the prices very close to the median. For plot 4, the use of color hue channel in addition to the color lumiance channel can be confusing to the user. Furthermore, the user might not know which point represent the Statue of Libery since all the attraction points are represented using a green point. One way to solve that problem is to use a different shape for each attraction point and add a comprehensive point shape legend to the plot.        